In [1]:
#import libraries
import sklearn 
import pandas as pd
import numpy as np
import nltk
import gensim

from gensim.models import Word2Vec


C:\Users\Student\Anaconda\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
#import datasets
df_train = pd.DataFrame.from_csv('C:/Users/Student/Desktop/labeledTrainData.tsv', sep='\t')
df_test = pd.DataFrame.from_csv('C:/Users/Student/Desktop/testData.tsv', sep='\t')
df_unlabel_train = pd.read_csv('C:/Users/Student/Desktop/unlabeledTrainData.tsv', sep='\t', error_bad_lines=False)


C:\Users\Student\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  
C:\Users\Student\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until
b'Skipping line 43043: expected 2 fields, saw 3\n'


In [4]:
df_train.head()

,sentiment,review
id,,
5814_8,1,With all this stuff going down at the moment w...
2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
7759_3,0,The film starts with a manager (Nicholas Bell)...
3630_4,0,It must be assumed that those who praised this...
9495_8,1,Superbly trashy and wondrously unpretentious 8...


# DATA CLEANING

REMOVAL OF PUNCTUATIONS AND CONVERTION OF ALPHABETS INTO LOWER CASE

In [5]:
df_train['review'] = df_train['review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_test['review'] = df_test['review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_unlabel_train['review'] = df_unlabel_train['review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_unlabel_train['review'] = df_unlabel_train['review'].str.replace('[^\w\s]','')
df_train['review'] = df_train['review'].str.replace('[^\w\s]','')
df_test['review'] = df_test['review'].str.replace('[^\w\s]','')

CLEANING OF REAR WORDS FROM THE DATA SET

In [6]:
#Rare words removal
freq1 = pd.Series(' '.join(df_train['review']).split()).value_counts()[-10:]
freq1 = list(freq1.index)
df_train['review'] = df_train['review'].apply(lambda x: " ".join(x for x in x.split() if x not in freq1))

freq1 = pd.Series(' '.join(df_test['review']).split()).value_counts()[-10:]
freq1 = list(freq1.index)
df_test['review'] = df_test['review'].apply(lambda x: " ".join(x for x in x.split() if x not in freq1))

freq1 = pd.Series(' '.join(df_unlabel_train['review']).split()).value_counts()[-10:]
freq1 = list(freq1.index)
df_unlabel_train['review'] = df_unlabel_train['review'].apply(lambda x: " ".join(x for x in x.split() if x not in freq1))

In [7]:
df_train.head()

,sentiment,review
id,,
5814_8,1,with all this stuff going down at the moment w...
2381_9,1,the classic war of the worlds by timothy hines...
7759_3,0,the film starts with a manager nicholas bell g...
3630_4,0,it must be assumed that those who praised this...
9495_8,1,superbly trashy and wondrously unpretentious 8...


In [8]:
df_train.shape

(25000, 2)

In [9]:
df_unlabel_train.shape

(49998, 2)

CREATION OF WORD EMBEDINGS

In [10]:
from gensim.models import Word2Vec

In [11]:
sentences = []
for i in range(0,25000):
    lsit = df_train.iloc[i]['review'].split()
    sentences.append(lsit)

In [12]:
for i in range(0,49998):
    lsit = df_unlabel_train.iloc[i]['review'].split()
    sentences.append(lsit)


In [13]:
import sys
model = Word2Vec(sentences=sentences, size=300, window=5, min_count=5, workers=4, sg=0)


Saving word2vec model in local system

In [14]:
model.save("word2vec.model")

In [15]:
model = gensim.models.Word2Vec.load("word2vec.model")

In [16]:
print(model)

Word2Vec(vocab=56387, size=300, alpha=0.025)


reference
https://radimrehurek.com/gensim/models/word2vec.html

In [17]:
vector = model.wv['computer']  # numpy vector of a word

In [18]:
vector.shape

(300,)

# Building a model

In [19]:
#time to train a CNN
import tensorflow as tf
import keras

Using TensorFlow backend.


In [20]:
#test-train split of the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train['review'], df_train['sentiment'], test_size=0.20, random_state=42)

In [21]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train)

In [22]:
k= 0
num_words = 60000
embedding_matrix = np.zeros((num_words, 300))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    if word in model:
        embedding_vector = model[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    else:
        k = k+1    

C:\Users\Student\Anaconda\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
C:\Users\Student\Anaconda\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [23]:
embedding_matrix.shape

(60000, 300)

In [ ]:
del model

In [31]:
sequences = tokenizer.texts_to_sequences(X_train)
sequences_val = tokenizer.texts_to_sequences(X_test)
x_train_seq = pad_sequences(sequences, maxlen=300)
x_val_seq = pad_sequences(sequences_val, maxlen=300)

In [32]:
len(sequences)

20000

In [33]:
len(x_train_seq[1])

300

In [34]:
#Define a lstm
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model


In [ ]:
from keras.layers import Conv1D
from keras.layers import GlobalMaxPooling1D
model = Sequential()
e = Embedding(60000, 300, weights=[embedding_matrix], input_length=300, trainable=False)
model.add(e)
model.add(Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1))
model.add(Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_test), epochs=3, batch_size=64, verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 300, 300)          18000000  
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 298, 100)          90100     
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 296, 100)          30100     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 100)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               25856     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 18,146,313
Trainable params: 146,313
Non-trainable params: 18,000,000
__________________________________________________________

In [65]:
y_pred_ = model.predict(x_val_seq, verbose=2)
y_new = np.zeros((len(y_pred),1))
for i in range(0,len(y_pred)):
    if y_pred[i]>0.5:
        y_new[i] = 1
    else:
        y_new[i] = 0


In [ ]:
print(sklearn.metrics.f1_score(y_train, y_new))

In [66]:
print(sklearn.metrics.f1_score(y_test, y_new))

0.7765363128491619


In [ ]:
y_out = model.predict(df_test['review'], verbose=2)
y = np.zeros((len(y_pred),1))
for i in range(0,len(y_pred)):
    if y_pred[i]>0.5:
        y[i] = 1
    else:
        y[i] = 0


In [ ]:
df=pd.DataFrame(y,index=df_test.index,columns=['sentiment'])
df.to_csv(output.csv)